In [3]:
import os
import json
import random
from uuid import uuid4
from datetime import datetime, timedelta, date

import psycopg2
from psycopg2.extras import execute_values

import pandas as pd

In [4]:
def get_conn():
    return psycopg2.connect(
        host=os.environ.get('PGHOST','localhost'),
        port=os.environ.get('PGPORT','5432'),
        dbname=os.environ.get('PGDATABASE','traveltech'),
        user=os.environ.get('PGUSER','postgres'),
        password=os.environ.get('PGPASSWORD','root')
    )

In [5]:
conn = get_conn()

In [27]:
AIRPORTS = [
    ("MAD","Adolfo Suárez Madrid–Barajas","Madrid","Spain","Europe/Madrid"),
    ("BCN","Barcelona–El Prat","Barcelona","Spain","Europe/Madrid"),
    ("CDG","Charles de Gaulle","Paris","France","Europe/Paris"),
    ("LHR","Heathrow","London","United Kingdom","Europe/London"),
    ("JFK","John F. Kennedy International","New York","USA","America/New_York"),
    ('PMI','Palma de Mallorca Airport','Palma','Spain','Europe/Madrid'),
    ('DXB','Dubai International Airport','Dubai','UAE','Asia/Dubai'),
    ('SCZ','Santa Cruz de la Sierra Airport','Santa Cruz','Bolivia','America/La_Paz')
]

travels_from_mad = [
    # (from_code, from_name, to_code, to_name, distance_km, flight_time_hours, [min_price_eur, max_price_eur])
    ("MAD", "Adolfo Suárez Madrid–Barajas", "BCN", "Barcelona–El Prat", 504, "1h 10m", [60, 180]),
    ("MAD", "Adolfo Suárez Madrid–Barajas", "CDG", "Charles de Gaulle", 1053, "2h 5m", [90, 250]),
    ("MAD", "Adolfo Suárez Madrid–Barajas", "LHR", "Heathrow", 1265, "2h 20m", [100, 280]),
    ("MAD", "Adolfo Suárez Madrid–Barajas", "PMI", "Palma de Mallorca Airport", 553, "1h 15m", [70, 200]),
    ("MAD", "Adolfo Suárez Madrid–Barajas", "JFK", "John F. Kennedy International", 5770, "8h 30m", [450, 1200]),
    ("MAD", "Adolfo Suárez Madrid–Barajas", "DXB", "Dubai International Airport", 5610, "7h 10m", [400, 1100]),
    ("MAD", "Adolfo Suárez Madrid–Barajas", "SCZ", "Santa Cruz de la Sierra Airport", 8850, "10 30m", [900, 1300]),
]

AIR_FARE_FAMILIES = [
    ("ECBASIC","Economy Basic",0,0.85,"NONREF"),
    ("ECFLEX","Economy Flex",1,1,"PARTIAL"),
    ("ECPLUS","Economy Plus",2,1.25,"PARTIAL"),
    ("BUSLIGHT","Business Light",3,1.8,"REF"),
    ("BUSFLEX","Business Flex",4,2.2,"FULL"),
    ("FIRST","First Class",5,3.5,"FULL"),
]
CABINS = ["ECONOMY","BUSINESS"]
STATUS = ["CONFIRMED","CANCELLED","AUTORIZED","FAILED"]

HOTELS = [
    # (hotel_id, name, chain, city, country, lat, lon, stars, [min_price_eur, max_price_eur])
    ("3aaf0cc6", "Hotel Centro", "UrbanStay", "Madrid", "Spain", 40.4168, -3.7038, 4, [120, 250]),
    ("4c9692b3", "Hotel Mar", "CoastInn", "Barcelona", "Spain", 41.3851, 2.1734, 4, [140, 280]),
    ("1ad4f8c1", "Le Meurice", "Hotels", "Paris", "France", 48.8658, 2.3275, 5, [600, 1500]),
    ("86a8b017", "Hotel Cap Rocat", "Hotel", "Palma", "Spain", 39.5331, 2.5819, 5, [400, 900]),
    ("a6065b00", "The Plaza", "FamousHotels", "New York", "USA", 40.7644, -73.9742, 5, [800, 2000]),
    ("fc55b3c4", "Burj Al Arab Jumeirah", "LuxuryStays", "Dubai", "UAE", 25.1412, 55.1853, 5, [1200, 3000]),
    ("8d1f74d7", "Los Tajibos Hotel", "Hotel", "Santa Cruz", "Bolivia", -17.7756, -63.1974, 3, [70, 150]),
    ("e5d181db", "The Savoy", "Hotel", "London", "United Kingdom", 51.5105, -0.1207, 5, [450, 1200]),
]

RATE_PLANS = ["RO","BB","HB"]
PAY_METHODS = ["CARD","PAYPAL"]
days = 8
random.seed(11)


In [56]:
events_list = [
    # (event_id, name, date, location, category, popularity, couple_price_range, notes)
    ("EVT001", "Mad Cool", "2026-08-28", "Madrid, Spain", "FESTIVAL", "HIGH", 
     "60-90 EUR", "Includes entry + transportation for 2"),
    
    ("EVT002", "Paris Fashion Week", "2026-09-23", "Paris, France", "FASHION", "HIGH",
     "180-1500 EUR", "2 show tickets + champagne reception"),
    
    ("EVT003", "Oktoberfest Palma", "2026-09-21", "Palma, Spain", "FESTIVAL", "VERY_HIGH",
     "100-200 EUR", "2 entry tickets + beer vouchers"),
    
    ("EVT004", "Hamilton Broadway", "2026-11-01", "New York, USA", "THEATER", "HIGH",
     "400-2000 USD", "2 premium seats, date varies by show"),
    
    ("EVT005", "Dubai Tour", "2026-12-15", "Dubai, UAE", "CULTURE", "HIGH",
     "350-450 AED", "2 tickets, VIP packages 500-2000 AED"),
    
    ("EVT006", "Coldplay Concert", "2026-05-24", "Barcelona, Spain", "CONCERT", "SOLD_OUT",
     "200-600 EUR", "2 general admission tickets"),
    
    ("EVT007", "Dubai Shopping Festival", "2027-03-15", "Dubai, UAE", "CULTURE", "HIGH",
     "600-1500 EUR", "All-inclusive 4-day package for 2"),
    
    ("EVT008", "Madrid Open Tennis", "2026-04-24", "Madrid, Spain", "SPORTS", "MEDIUM",
     "100-800 EUR", "2 tickets + parking, varies by session"),
    
    ("EVT009", "London Pass", "2026-04-21", "London, UK", "CULTURE", "VERY_HIGH",
     "200-400 GBP", "2 spectator grandstand seats + lunch"),
    
    ("EVT010", "BCN Tour and Flavors", "2026-10-07", "Barcelona, Spain", "FOOD", "MEDIUM",
     "200-400 EUR", "2 passes + gourmet dinner")
]

In [101]:
AIRCRAFT_FLEET = [
    # Short-haul (domestic/regional)
    {"manufacturer": "Airbus", "model": "A320-200", "seats": 180, "range_km": 6100, "type": "SHORT_HAUL"},
    {"manufacturer": "Boeing", "model": "737-800", "seats": 189, "range_km": 5765, "type": "SHORT_HAUL"},
    {"manufacturer": "Airbus", "model": "A321neo", "seats": 240, "range_km": 7400, "type": "SHORT_MEDIUM"},
    {"manufacturer": "Embraer", "model": "E190", "seats": 114, "range_km": 4445, "type": "REGIONAL"},
    
    # Medium-haul (European)
    {"manufacturer": "Airbus", "model": "A320neo", "seats": 195, "range_km": 6850, "type": "MEDIUM_HAUL"},
    {"manufacturer": "Boeing", "model": "737 MAX 8", "seats": 210, "range_km": 6570, "type": "MEDIUM_HAUL"},
    
    # Long-haul (transatlantic/intercontinental)
    {"manufacturer": "Airbus", "model": "A330-300", "seats": 300, "range_km": 11750, "type": "LONG_HAUL"},
    {"manufacturer": "Boeing", "model": "787-9 Dreamliner", "seats": 296, "range_km": 14140, "type": "LONG_HAUL"},
    {"manufacturer": "Airbus", "model": "A350-900", "seats": 325, "range_km": 15000, "type": "ULTRA_LONG_HAUL"},
    {"manufacturer": "Boeing", "model": "777-300ER", "seats": 396, "range_km": 14685, "type": "ULTRA_LONG_HAUL"},
    
    # Business/First class focused
    {"manufacturer": "Airbus", "model": "A321LR", "seats": 206, "range_km": 8700, "type": "PREMIUM"},
    {"manufacturer": "Boeing", "model": "767-300ER", "seats": 269, "range_km": 11370, "type": "PREMIUM"},
]

# Define airport distances from MAD (in km)
DESTINATION_DISTANCES = {
    "BCN": 504,      # Domestic
    "PMI": 553,      # Domestic (island)
    "CDG": 1053,     # Short-haul European
    "LHR": 1265,     # Short-haul European
    "DXB": 5610,     # Long-haul Middle East
    "JFK": 5770,     # Long-haul Transatlantic
    "SCZ": 8850,     # Ultra-long-haul (with stop)
}

In [22]:
cursor = conn.cursor()
cursor.executemany(
"INSERT INTO travel.airport(airport_code,name,city,country,tz) VALUES (%s,%s,%s,%s,%s) ON CONFLICT "
"(airport_code) DO NOTHING", AIRPORTS)
conn.commit() 
cursor.close()

In [19]:
cursor = conn.cursor()
cursor.executemany(
    "INSERT INTO travel.fare_families(fare_family_id,name,bags_included,change_penalty,refundability) VALUES (%s,%s,%s,%s,%s) ON CONFLICT (fare_family_id) DO NOTHING",
    AIR_FARE_FAMILIES)
conn.commit() 
cursor.close()

In [26]:
conn.commit() 
cursor.close()

In [12]:
def seed_air_reference(conn):
    with conn.cursor() as cur:
        cur.executemany(
        "INSERT INTO travel_air.airports(airport_code,name,city,country,tz) VALUES (%s,%s) ON CONFLICT "
        "(airport_code) DO NOTHING", AIRPORTS)
        cur.executemany(
        "INSERT INTO travel_air.fare_families(fare_family_id,name,bags_included,change_penalty,refundability) " \
        "VALUES (%s,%s,%s,%s,%s) ON CONFLICT (fare_family_id) DO NOTHING",
        AIR_FARE_FAMILIES
        )

In [ ]:
def seed_air_reference(conn):
    with conn.cursor() as cur:
        execute_values(cur,
            "INSERT INTO travel_air.airports(airport_code,name,city,country,tz) VALUES %s ON CONFLICT (airport_code) DO NOTHING",
            AIRPORTS
        )
        execute_values(cur,
            "INSERT INTO travel_air.fare_families(fare_family_id,name,bags_included,change_penalty,refundability) VALUES %s ON CONFLICT (fare_family_id) DO NOTHING",
            AIR_FARE_FAMILIES
        )

8

In [39]:
from datetime import datetime, timedelta
import random

FLIGHT_DURATIONS = {
    ("MAD", "CDG"): (1.5, 2),      # Madrid to Paris: 1h45m-2h
    ("BCN", "LHR"): (2, 2.5),       # Barcelona to London: 2h-2h15m
    ("MAD", "BCN"): (1, 1.5),       # Madrid to Barcelona: 1h10m
    ("MAD", "PMI"): (1, 1.5),       # Madrid to Palma de Mallorca: 1h15m
    ("MAD", "DXB"): (7, 7.5),       # Madrid to Dubai: ~7h
    ("MAD", "SCZ"): (11, 12.5),     # Madrid to Santa Cruz, Bolivia: ~11h (with connections)
    ("MAD", "JFK"): (8.5, 9)        # Madrid to New York: 8h30m-9h
}

# Realistic departure times based on typical flight schedules
DEPARTURE_WINDOWS = {
    "short_haul": [(6, 10), (14, 18), (19, 22)],  # Short flights: morning, afternoon, evening
    "long_haul": [(10, 13), (14, 17), (18, 24)],   # Long flights: day & overnight
}

# Get route type based on duration
def get_route_type(duration_range):
    avg_duration = sum(duration_range) / 2
    return "long_haul" if avg_duration > 4 else "short_haul"

now = datetime.now().replace(microsecond=0)
ods = [("MAD","CDG"),("BCN","LHR"),("MAD","BCN"),("MAD","PMI"),("MAD","DXB"),("MAD","SCZ"),("MAD","JFK")]
flights = []
days = 8

for origin, destination in ods:
    duration_range = FLIGHT_DURATIONS.get((origin, destination), (2, 3))
    
    # Determine departure time windows based on flight length
    route_type = get_route_type(duration_range)
    departure_windows = DEPARTURE_WINDOWS[route_type]
    print(origin, destination, route_type)
    for day_offset in range(days):
        window = random.choice(departure_windows)
        print("Selected window:", window)
        start_hour, end_hour = window
        
        dep_date = now + timedelta(days=day_offset)
        dep_hour = random.randint(start_hour, end_hour - 1)
        dep_minute = random.choice([0, 15, 30, 45])
        
        # Handle overnight flights (hour 22-1)
        if end_hour < start_hour:  # Overnight window (e.g., 22-1)
            dep_hour = random.randint(start_hour, 23)
            if dep_hour >= start_hour and dep_hour <= 23:
                pass
            else:
                dep_hour = 0  # Handle the wrap-around
        
        dep = dep_date.replace(hour=dep_hour, minute=dep_minute, second=0)
        
        min_dur, max_dur = duration_range
        flight_hours = random.uniform(min_dur, max_dur)
        flight_timedelta = timedelta(hours=flight_hours)
        arr = dep + flight_timedelta
        
        flight_num = f"{random.randint(100, 999)}"
        flight_id = f"IB{flight_num}-{dep.date()}-{origin}-{destination}"
        
        status_weights = ["SCHEDULED"] * 8 + ["DELAYED"] * 2
        status = random.choice(status_weights)
        
        flights.append((
            flight_id, "IB", flight_num, dep.date(), 
            origin, destination, dep.isoformat() + "Z", 
            arr.isoformat() + "Z", status
        ))

flights.sort(key=lambda x: x[6])

# Print a sample for verification
print(f"Generated {len(flights)} flights")
print("\nSample flights:")
for i, flight in enumerate(flights[:3]):
    fid, airline, fn, date, o, d, dep, arr, status = flight
    print(f"{i+1}. {fid}: {o}→{d} | Dep: {dep} | Arr: {arr} | Status: {status}")

MAD CDG short_haul
Selected window: (14, 18)
Selected window: (14, 18)
Selected window: (14, 18)
Selected window: (6, 10)
Selected window: (14, 18)
Selected window: (19, 22)
Selected window: (6, 10)
Selected window: (6, 10)
BCN LHR short_haul
Selected window: (6, 10)
Selected window: (14, 18)
Selected window: (14, 18)
Selected window: (19, 22)
Selected window: (14, 18)
Selected window: (14, 18)
Selected window: (6, 10)
Selected window: (14, 18)
MAD BCN short_haul
Selected window: (6, 10)
Selected window: (19, 22)
Selected window: (19, 22)
Selected window: (6, 10)
Selected window: (14, 18)
Selected window: (6, 10)
Selected window: (19, 22)
Selected window: (6, 10)
MAD PMI short_haul
Selected window: (19, 22)
Selected window: (14, 18)
Selected window: (6, 10)
Selected window: (6, 10)
Selected window: (19, 22)
Selected window: (6, 10)
Selected window: (14, 18)
Selected window: (6, 10)
MAD DXB long_haul
Selected window: (10, 13)
Selected window: (10, 13)
Selected window: (10, 13)
Selected 

In [42]:
cursor = conn.cursor()
cursor.executemany(
    """INSERT INTO travel.flights(flight_id,carrier_code,flight_number,flight_date,origin,destination,
    departure_ts,arrival_ts,status) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s) ON CONFLICT (flight_id) DO NOTHING""",
    flights
)
conn.commit()
cursor.close()

In [71]:
inventories, prices = [], []
for (fid, *_rest) in flights:
    for cabin in CABINS:
        for ff in AIR_FARE_FAMILIES:
            inv_id = str(uuid4())[:8]
            seats = random.randint(5, 100 if cabin=="ECONOMY" else 20)
            ts = datetime.now().isoformat()+"Z"
            inventories.append((inv_id,fid,cabin,ff[0],seats,ts))
            base = random.randint(40,160)
            if cabin == "BUSINESS":
                base *= 2.5
            if ff[0]=="ECBASIC":
                base *= 0.85
            elif ff[0]=="BUSFLEX":
                base *= 1.35
            taxes = round(base*0.21,2)
            total = round(base+taxes,2)
            prices.append((str(uuid4())[:8],inv_id,"EUR",round(base,2),taxes,total,(datetime.now()+timedelta(minutes=30)).isoformat()+"Z"))


In [75]:
print(len(inventories), len(prices))
print(inventories[0], "\n", prices[0])

336 336
('cf566e91', 'IB166-2026-01-20-MAD-BCN', 'ECONOMY', 'ECBASIC', 65, '2026-01-20T18:28:27.371835Z') 
 ('a4c2a292', 'cf566e91', 'EUR', 61.2, 12.85, 74.05, '2026-01-20T18:58:27.371873Z')


In [44]:
cursor = conn.cursor()
cursor.executemany(
    """INSERT INTO travel.flight_inventory(inventory_id,flight_id,cabin_class,fare_family_id,seats_available,last_updated_ts) 
    VALUES (%s,%s,%s,%s,%s,%s) ON CONFLICT (inventory_id) DO NOTHING""",
    inventories
)
cursor.executemany(
    """INSERT INTO travel.prices(price_id,inventory_id,currency,base_fare,taxes,total_fare,valid_until_ts) 
    VALUES (%s,%s,%s,%s,%s,%s,%s) ON CONFLICT (price_id) DO NOTHING""",
    prices
)
conn.commit()
cursor.close()

In [10]:
cursor = conn.cursor()
cursor.executemany(
    """INSERT INTO travel.hotels(hotel_id,name,chain,city,country,lat,lon,stars) 
    VALUES (%s,%s,%s,%s,%s,%s,%s,%s) ON CONFLICT (hotel_id) DO NOTHING""",
    HOTELS
)
conn.commit()
cursor.close()

In [32]:
with conn.cursor() as cursor:
    cursor.execute("""SELECT * FROM travel.hotels""")
    data = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]
    df = pd.DataFrame(data, columns=column_names)

In [56]:
l_hotels = df['hotel_id']

In [39]:
start = date.today()
inv_rows, price_rows = [], []
for h in l_hotels:
    for i in range(days):
        dt = start + timedelta(days=i)
        total = random.randint(10, 40)
        sold = random.randint(0, total-1)
        inv_rows.append((str(uuid4())[:8], h, dt, total, sold, random.randint(0,3)))

print(len(inv_rows), inv_rows[0])

cursor = conn.cursor()
cursor.executemany(
    """INSERT INTO travel.inventory(inventory_id,hotel_id,date,allotment_total,
    allotment_sold,release_period_days) VALUES (%s,%s,%s,%s,%s,%s) 
    ON CONFLICT (inventory_id) DO NOTHING""",
    inv_rows
)
conn.commit()
cursor.close()

64 ('297e3f78', '3aaf0cc6', datetime.date(2026, 1, 20), 25, 17, 0)


In [ ]:
names = ["Daniel", "Elsa", "Maria", "Sergio", "Carlos", "Macià", "Jose", "Nacho"]
last_names = ['Vedia', 'Nunes', 'Bueno', 'Guillem', 'Novoa', 'Mut', 'Yaturri', 'Garrón']

# Marta Rodríguez
# Isabel Bolero
# Gills Winthere

rows = []

name_lastname_pairs = list(zip(names, last_names))

email_domains = ["demo.com", "mail.com"]

LOYALTY_PROBABILITY = 0.7

for i in range(40):
    for name, last_name in name_lastname_pairs:
        cid = str(uuid4())[:8]
        
        email_domain = random.choice(email_domains)
        email_num = random.randint(1, 999)
        email = f"{name.lower()}.{last_name.lower()}{i}{email_num}@{email_domain}"
        
        full_name = f"{name} {last_name}"
        
        phone_prefix = random.choice(["+34 6", "+34 7"])
        phone_number = f"{phone_prefix}{random.randint(10, 99)} {random.randint(100, 999)} {random.randint(100, 999)}"
        
        loyalty_code = None
        if random.random() < LOYALTY_PROBABILITY:
            loyalty_code = f"LOY{random.randint(100000, 999999)}"
        
        rows.append((
            cid,
            email,
            full_name,
            phone_number,
            loyalty_code
        ))


print(f"Generated {len(rows)} customer records")
print("\nSample records:")
for i, row in enumerate(rows[:5]):
    print(f"{i+1}. ID: {row[0]}, Name: {row[2]}, Email: {row[1]}")
    
loyal_customers = sum(1 for row in rows if row[4] is not None)
print(f"\nCustomers with loyalty codes: {loyal_customers} ({loyal_customers/len(rows)*100:.1f}%)")

cursor = conn.cursor()
cursor.executemany(
    """INSERT INTO travel.customers(customer_id,email,name,phone,loyalty_id) 
    VALUES (%s,%s,%s,%s,%s) ON CONFLICT (customer_id) DO NOTHING""",
    rows
)
conn.commit()
cursor.close()

Generated 320 customer records

Sample records:
1. ID: b0595b9e, Name: Daniel Vedia, Email: daniel.vedia0735@demo.com
2. ID: 47a5134a, Name: Elsa Nunes, Email: elsa.nunes0697@mail.com
3. ID: b6eceef7, Name: Maria Bueno, Email: maria.bueno0907@demo.com
4. ID: 96dc0e89, Name: Sergio Guillem, Email: sergio.guillem0472@demo.com
5. ID: 371f8a70, Name: Carlos Novoa, Email: carlos.novoa0229@demo.com

Customers with loyalty codes: 224 (70.0%)


In [45]:
with conn.cursor() as cursor:
    cursor.execute("""SELECT * FROM travel.customers""")
    data = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]
    df = pd.DataFrame(data, columns=column_names)

l_customer = df['customer_id'].tolist()

with conn.cursor() as cursor:
    cursor.execute("""SELECT * FROM travel.hotels""")
    data = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]
    df = pd.DataFrame(data, columns=column_names)

l_hotels = df['hotel_id']

In [46]:
bookings =[]
for _ in range(220):
    bid = str(uuid4())[:8]
    status = random.choice(['PENDING_PAYMENT','CONFIRMED','CANCELLED','TICKETED'])
    type_b = random.choice(['AIR','HOTEL'])
    code = f"PNR{uuid4().hex[:6].upper()}"
    cust = random.choice(l_customer)
    hotel = random.choice(l_hotels)
    price = round(random.randint(40,160)*1.21,2)
    bookings.append((bid, cust, hotel,type_b, status, code, 'EUR', float(price), datetime.now()))

cursor = conn.cursor()
cursor.executemany(
    """INSERT INTO travel.bookings(booking_id,customer_id,hotel_id,booking_type,status,
    confirmation_code,currency,total_amount,created_at) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s) 
    ON CONFLICT (booking_id) DO NOTHING""", bookings)
conn.commit()
cursor.close()

In [ ]:
from datetime import datetime, timedelta
import random
from uuid import uuid4

BASE_ECONOMY_PRICES = {
    ("MAD", "BCN"): (60, 120),      # Short domestic: 60-120€
    ("MAD", "PMI"): (70, 140),      # Domestic island: 70-140€
    ("MAD", "CDG"): (90, 180),      # Short-haul EU: 90-180€
    ("MAD", "LHR"): (100, 200),     # Short-haul EU: 100-200€
    ("MAD", "DXB"): (400, 800),     # Long-haul M.East: 400-800€
    ("MAD", "JFK"): (450, 900),     # Long-haul NA: 450-900€
    ("MAD", "SCZ"): (700, 1400),    # Long-haul SA (connects): 700-1400€
}

CABIN_MULTIPLIERS = {
    "ECONOMY": 1.0,
    "PREMIUM_ECONOMY": 1.8,
    "BUSINESS": 3.0,
    "FIRST": 5.0
}

FARE_FAMILY_MODIFIERS = {
    "ECBASIC": 0.85,      # Basic economy - cheapest, no changes
    "ECFLEX": 1.0,        # Standard economy
    "ECPLUS": 1.25,       # Economy plus (extra legroom, priority)
    "BUSLIGHT": 1.8,      # Business light (restricted)
    "BUSFLEX": 2.2,       # Fully flexible business
    "FIRST": 3.5          # First class
}

TAX_RATES = {
    "Spain": 0.21,        # IVA 21%
    "France": 0.20,       # TVA 20%
    "United Kingdom": 0.20, # VAT 20%
    "USA": 0.075,         # Average US sales tax ~7.5%
    "UAE": 0.05,          # VAT 5%
    "Bolivia": 0.13,      # IVA 13%
}

airport_info = {code: (name, city, country, tz) for code, name, city, country, tz in AIRPORTS}

inventories, prices = [], []

for flight in flights:
    fid, airline, flight_num, flight_date, origin, destination, dep_time, arr_time, status = flight
    
    if origin != "MAD" or destination not in airport_info:
        continue
    
    # Get country for tax calculation
    dest_country = airport_info[destination][2]
    tax_rate = TAX_RATES.get(dest_country, 0.20)  # Default 20% if country not found
    
    base_min, base_max = BASE_ECONOMY_PRICES.get((origin, destination), (100, 200))
    
    for cabin in CABINS:
        for ff_code, ff_name, id, nr, ref in AIR_FARE_FAMILIES:  # Assuming AIR_FARE_FAMILIES is list of tuples
            
            inv_id = str(uuid4())[:8]
            
            # Calculate available seats based on cabin
            if cabin == "ECONOMY":
                seats = random.randint(20, 180)
            elif cabin == "BUSINESS":
                seats = random.randint(6, 16)
              
            # Add timestamp
            ts = datetime.now().isoformat() + "Z"
            
            inventories.append((inv_id, fid, cabin, ff_code, seats, ts))
            
            # --- PRICE CALCULATION ---
            base_economy = random.uniform(base_min, base_max)
            
            # 2. Apply cabin multiplier
            cabin_multiplier = CABIN_MULTIPLIERS.get(cabin, 1.0)
            price = base_economy * cabin_multiplier
            
            # 3. Apply fare family modifier
            fare_modifier = FARE_FAMILY_MODIFIERS.get(ff_code, 1.0)
            price *= fare_modifier
            
            # 4. Dynamic pricing factors
            # a. Days before departure (closer = more expensive)
            days_to_departure = (datetime.fromisoformat(dep_time[:-1]) - datetime.now()).days
            if days_to_departure < 7:
                price *= random.uniform(1.3, 2.0)  # 30-100% increase last week
            elif days_to_departure < 14:
                price *= random.uniform(1.1, 1.4)  # 10-40% increase last 2 weeks
            
            # b. Weekend/peak travel premium
            dep_day = datetime.fromisoformat(dep_time[:-1]).weekday()
            if dep_day >= 4:  # Friday, Saturday, Sunday
                price *= random.uniform(1.1, 1.25)
            
            # c. Seasonal demand (summer/holiday premium)
            dep_month = datetime.fromisoformat(dep_time[:-1]).month
            if dep_month in [6, 7, 8, 12]:  # Summer & Christmas
                price *= random.uniform(1.15, 1.4)
            
            # Round base price
            base_price = round(price, 2)
            
            # Calculate taxes and total
            taxes = round(base_price * tax_rate, 2)
            total_price = round(base_price + taxes, 2)
            
            # Generate price ID and validity timestamp (prices valid for 30 min)
            price_id = str(uuid4())[:8]
            valid_until = (datetime.now() + timedelta(minutes=30)).isoformat() + "Z"
            
            prices.append((
                price_id,
                inv_id,
                "EUR",
                base_price,
                taxes,
                total_price,
                valid_until
            ))

print(f"Generated {len(inventories)} inventory items")
print(f"Generated {len(prices)} price points")
print(f"\nSample price calculations for flights from MAD:")

for i, (inv, price) in enumerate(zip(inventories[:5], prices[:5])):
    inv_id, fid, cabin, ff, seats, _ = inv
    price_id, _, currency, base, taxes, total, valid = price
    print(f"\n{i+1}. {fid} | Cabin: {cabin} | Fare: {ff}")
    print(f"   Seats: {seats} | Base: {base}{currency} | Taxes: {taxes}{currency} | Total: {total}{currency}")
    print(f"   Valid until: {valid}")

print(f"\n{'='*60}")
print("Average prices by cabin class:")
cabins_data = {}
for inv, price in zip(inventories, prices):
    cabin = inv[2]
    total_price = price[5]
    if cabin not in cabins_data:
        cabins_data[cabin] = []
    cabins_data[cabin].append(total_price)

for cabin, prices_list in cabins_data.items():
    avg_price = sum(prices_list) / len(prices_list)
    print(f"{cabin:20} avg: {avg_price:.2f}€ (min: {min(prices_list):.2f}€, max: {max(prices_list):.2f}€)")

Generated 576 inventory items
Generated 576 price points

Sample price calculations for flights from MAD:

1. IB197-2026-01-22-MAD-BCN | Cabin: ECONOMY | Fare: ECBASIC
   Seats: 107 | Base: 141.89EUR | Taxes: 29.8EUR | Total: 171.69EUR
   Valid until: 2026-01-22T21:30:01.254317Z

2. IB197-2026-01-22-MAD-BCN | Cabin: ECONOMY | Fare: ECFLEX
   Seats: 34 | Base: 208.03EUR | Taxes: 43.69EUR | Total: 251.72EUR
   Valid until: 2026-01-22T21:30:01.254355Z

3. IB197-2026-01-22-MAD-BCN | Cabin: ECONOMY | Fare: ECPLUS
   Seats: 23 | Base: 179.61EUR | Taxes: 37.72EUR | Total: 217.33EUR
   Valid until: 2026-01-22T21:30:01.254436Z

4. IB197-2026-01-22-MAD-BCN | Cabin: ECONOMY | Fare: BUSLIGHT
   Seats: 71 | Base: 310.59EUR | Taxes: 65.22EUR | Total: 375.81EUR
   Valid until: 2026-01-22T21:30:01.254501Z

5. IB197-2026-01-22-MAD-BCN | Cabin: ECONOMY | Fare: BUSFLEX
   Seats: 102 | Base: 266.93EUR | Taxes: 56.06EUR | Total: 322.99EUR
   Valid until: 2026-01-22T21:30:01.254525Z

Average prices by cabin

4:3 (todo el sensor), grid line, X2 o X3 solo zoom (retrato)

In [37]:
print(len(inventories), len(prices))
print(inventories[0], "\n", prices[0])

6000 6000
('1d5aac21', 'IB130-2026-02-14-MAD-DXB', 'ECONOMY', 'ECBASIC', 139, '2026-01-22T20:50:51.533838Z') 
 ('8a28da50', '1d5aac21', 'EUR', 514.28, 25.71, 539.99, '2026-01-22T21:20:51.533924Z')


In [8]:
with conn.cursor() as cursor:
    cursor.execute("""SELECT * FROM travel.prices""")
    data = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]
    df_p = pd.DataFrame(data, columns=column_names)

with conn.cursor() as cursor:
    cursor.execute("""SELECT * FROM travel.flight_inventory""")
    data = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]
    df_f = pd.DataFrame(data, columns=column_names)

with conn.cursor() as cursor:
    cursor.execute("""SELECT * FROM travel.customers""")
    data = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]
    df_c = pd.DataFrame(data, columns=column_names)

with conn.cursor() as cursor:
    cursor.execute("""SELECT * FROM travel.flights""")
    data = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]
    df_fl = pd.DataFrame(data, columns=column_names)

with conn.cursor() as cursor:
    cursor.execute("""SELECT * FROM travel.bookings""")
    data = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]
    df_b = pd.DataFrame(data, columns=column_names)

with conn.cursor() as cursor:
    cursor.execute("""SELECT * FROM travel.hotel_prices""")
    data = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]
    df_hp = pd.DataFrame(data, columns=column_names)

In [ ]:
bookings_i = []
for pr in range(576):
    bid = str(uuid4())[:8]
    price_id = df_p['price_id'].iloc[pr]
    hotel_id = random.choice(l_hotels)
    flight_id = df_fl['flight_id'].iloc[random.randint(0, len(df_fl)-1)]
    cabin = random.choice(CABINS) 
    fare = random.choice(AIR_FARE_FAMILIES)[0]
    
    bookings_i.append((bid, cust_id, flight_id, cabin, fare, price_id))  

cursor = conn.cursor()
cursor.executemany(
    """INSERT INTO travel.booking_items(booking_item_id,customer_id,flight_id,cabin_class,
    fare_family_id,price_id) VALUES (%s,%s,%s,%s,%s,%s) ON CONFLICT (booking_item_id) DO NOTHING""",
    bookings_i)
conn.commit() 
cursor.close()

In [8]:
from datetime import datetime, timedelta
import random

# One-liner to generate a list of random datetimes
random_dates = [
    datetime.now() + timedelta(
        days=random.randint(0, 14),
        hours=random.randint(0, 23),
        minutes=random.randint(0, 59),
        seconds=random.randint(0, 59)
    )
    for _ in range(15)
]

random_dates.sort()
# for dt in random_dates:
#     print(dt.strftime("%Y-%m-%d %H:%M:%S"))

In [87]:
hotel_pr = [
    (str(uuid4())[:8],
    random.choice(l_hotels),
    random_dates[random.randint(0, len(random_dates)-1)].strftime("%Y-%m-%d %H:%M:%S"),
    'EUR',
    float(df_b[(df_b['hotel_id'] == random.choice(l_hotels)) & 
        (df_b['booking_type'] == 'HOTEL')]['total_amount'].mean() 
        if not df_b.empty else random.uniform(80, 300)),
        random.choices(['TRUE', 'FALSE'])[0],
        df_b[df_b['hotel_id'] == random.choice(l_hotels)]['booking_id'].iloc[0] 
    )
    for i in range(440)
]

cursor = conn.cursor()
cursor.executemany(
    """INSERT INTO travel.hotel_prices(rate_plan_id,hotel_id,date,currency,price_per_night,
    tax_included,booking_id) VALUES (%s,%s,%s,%s,%s,%s,%s) ON CONFLICT (rate_plan_id) DO NOTHING""",
    hotel_pr)
conn.commit() 
cursor.close()

In [9]:
pr_b = df_b['total_amount'].to_list()
pr_f = df_p['total_fare'].to_list()
l_c = df_c['customer_id'].to_list()


seqs = [pr_b, pr_f]

In [26]:
pays = [
    (str(uuid4())[:8],
    random.choice(PAY_METHODS),
    random.choice(random.choices(seqs, weights=map(len, seqs))[0]),
    'EUR',
    random.choice(STATUS),
    str(uuid4())[:8],
    random_dates[random.randint(0, len(random_dates)-1)].strftime("%Y-%m-%d %H:%M:%S"),
    random.choice(l_c) 
    )
    for i in range(560)
]

cursor = conn.cursor()
cursor.executemany(
    """INSERT INTO travel.payments(payment_id,method,amount,currency,status,
authorization_code,created_at,customer_id) VALUES (%s,%s,%s,%s,%s,%s,%s,%s) ON CONFLICT 
(payment_id) DO NOTHING""", pays)
conn.commit() 
cursor.close()

In [138]:
df_fl[~df_fl['aircraft_code'].isna()]

,flight_id,carrier_code,flight_number,flight_date,origin,destination,departure_ts,arrival_ts,aircraft_code,status


In [44]:
ticket = [
    (str(uuid4())[:8],
    random.choice(df_fl['flight_id'].to_list()),
    random_dates[random.randint(0, len(random_dates)-1)].strftime("%Y-%m-%d %H:%M:%S"),
    str(uuid4())[:8],
    random.choice(['ON_SITE','ONLINE','PENDING']),
    random.choice(l_c)
    )
    for i in range(440)
]

cursor = conn.cursor()
cursor.executemany(
    """INSERT INTO travel.tickets(ticket_id,flight_id,issued_ts,eticket_number,status,customer_id) 
    VALUES (%s,%s,%s,%s,%s,%s) ON CONFLICT (ticket_id) DO NOTHING""", ticket)
conn.commit() 
cursor.close()

In [44]:
# events_list[random.randint(0, len(events_list)-1)][6].split("-")
import re

price_str = "350-450 AED"
numbers = [int(num) for num in re.findall(r'\d+', price_str)]
numbers

[350, 450]

In [32]:
event = [
    (str(uuid4())[:8],
     events_list[random.randint(0, len(events_list)-1)][0],
     random_dates[random.randint(0, len(random_dates)-1)].strftime("%Y-%m-%d %H:%M:%S"),
     'null',
     'null',
     random.choice(l_c),
     random.choice(l_c), 0, 'null'
    )
    for i in range(720)
]

cursor = conn.cursor()
cursor.executemany(
    """INSERT INTO travel.events(event_id,entity_id,event_ts,event_name,payload,customer_id,
     customer_id2, price_event, city) 
    VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s) ON CONFLICT (event_id) DO NOTHING""", event)
conn.commit() 
cursor.close()

In [69]:
# for i in range(len(events_list)):
print("""UPDATE travel.events SET name = %s WHERE entity_id = %s""",
    (events_list[5][3], events_list[5][0]))

UPDATE travel.events SET name = %s WHERE entity_id = %s ('Barcelona, Spain', 'EVT006')


In [ ]:
import re

price_str = "350-450 AED"
numbers = [int(num) for num in re.findall(r'\d+', price_str)]
numbers

In [87]:
with conn.cursor() as cursor:
    cursor.execute("""SELECT * FROM travel.events""")
    data = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]
    df_e = pd.DataFrame(data, columns=column_names)

l_event = df_e['event_id']

In [88]:
cursor = conn.cursor()

for i in range(len(events_list)):
    price = [int(num) for num in re.findall(r'\d+', events_list[i][6])]
    for j in range(len(df_e)):
        if 'event_id' in df_e.columns:
            event_id = df_e.at[j, 'event_id']
    cursor.execute("""UPDATE travel.events SET price_event = %s WHERE event_id = %s""",
    (random.choice(price), event_id))

conn.commit()
cursor.close()

In [ ]:
cursor = conn.cursor()

for i in range(len(df_fl)):
    destination = df_fl.at[i, 'destination']
    
    if destination in list(ai_mp.keys()):
        # Get random aircraft for this destination
        aircraft_code = random.choice(ai_mp[destination])
        
        # Assuming you have a flight_id column to update specific flight
        if 'flight_id' in df_fl.columns:
            flight_id = df_fl.at[i, 'flight_id']
            cursor.execute(
                """UPDATE travel.flights SET aircraft_code = %s WHERE flight_id = %s""",
                (aircraft_code, flight_id)
            )
        
conn.commit()
cursor.close()

In [ ]:
routes = [
    (str(uuid4())[:8],
     'MAD',
    for i in range(len(travels_from_mad)):
        travels_from_mad[i][2]
        travels_from_mad[i][3]
    )
    for i in range(720)
]

cursor = conn.cursor()
cursor.executemany(
    """INSERT INTO travel.events(event_id,entity_id,event_ts,event_name,payload,customer_id) 
    VALUES (%s,%s,%s,%s,%s,%s) ON CONFLICT (event_id) DO NOTHING""", event)
conn.commit() 
cursor.close()

In [100]:
routes = [
    (
        str(uuid4())[:8],                    # route_id
        'MAD',                               # origin_code
        i[2],                           # dest_code
        i[4]                            # distance_km
    )
    for i in travels_from_mad # Random travel for each iteration
]

cursor = conn.cursor()
cursor.executemany(
    """INSERT INTO travel.routes(route_id,origin,destination,distance_km) 
    VALUES (%s,%s,%s,%s) ON CONFLICT (route_id) DO NOTHING""", routes)
conn.commit()
cursor.close()

In [122]:
def select_aircraft_for_route(destination, is_business_heavy=False):
    distance = DESTINATION_DISTANCES.get(destination, 1000)
    
    # Filter aircraft by range and type
    if distance <= 1500:
        # Domestic/short European routes
        if is_business_heavy:
            candidates = [a for a in AIRCRAFT_FLEET if a["type"] in ["REGIONAL", "SHORT_HAUL", "PREMIUM"] and a["range_km"] >= distance]
        else:
            candidates = [a for a in AIRCRAFT_FLEET if a["type"] in ["REGIONAL", "SHORT_HAUL", "SHORT_MEDIUM"] and a["range_km"] >= distance]
    
    elif distance <= 4500:
        # Medium-haul European/North Africa
        candidates = [a for a in AIRCRAFT_FLEET if a["type"] in ["MEDIUM_HAUL", "SHORT_MEDIUM", "A320neo"] and a["range_km"] >= distance]
    
    elif distance <= 8000:
        # Long-haul (Middle East, East Coast USA)
        candidates = [a for a in AIRCRAFT_FLEET if a["type"] in ["LONG_HAUL", "A321LR"] and a["range_km"] >= distance]
    
    else:
        # Ultra-long-haul (Americas, Asia)
        candidates = [a for a in AIRCRAFT_FLEET if a["type"] in ["ULTRA_LONG_HAUL", "LONG_HAUL"] and a["range_km"] >= distance]
    
    # If no candidates found, use any aircraft with sufficient range
    if not candidates:
        candidates = [a for a in AIRCRAFT_FLEET if a["range_km"] >= distance]
    
    return candidates

def generate_aircraft_table(destination):
    aircraft_table = []
    
    business_heavy_destinations = ["JFK", "BCN", "LHR", "CDG"]
    is_business_heavy = destination in business_heavy_destinations 
    
    aircraft = select_aircraft_for_route(destination, is_business_heavy)
    aircraft_table = pd.DataFrame(aircraft)
    aircraft_table['destination'] = destination
      
    return aircraft_table

# Generate aircraft data
aircraft_data = generate_aircraft_table('PMI')

In [127]:
aircraft_data = pd.DataFrame()
for d in list(DESTINATION_DISTANCES.keys()):
    table = generate_aircraft_table(d)
    aircraft_data = pd.concat([aircraft_data, table], ignore_index=True)

In [152]:
aircraft_data

with conn.cursor() as cursor:
    cursor.execute("""SELECT * FROM travel.aircraft""")
    data = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]
    df_ai = pd.DataFrame(data, columns=column_names)

In [172]:
air_df= pd.concat([df_ai, aircraft_data[['destination']]], axis=1)
air_df.head(2)

,aircraft_code,manufacturer,model,seats_total,destination
0,0c9cd04f,Airbus,A320-200,180,BCN
1,1748ee67,Boeing,737-800,189,BCN


In [171]:
df_fl.head(2)

flight_id       IB197-2026-01-22-MAD-BCN
carrier_code                          IB
Name: 1, dtype: object

In [225]:
ai_mp = air_df.groupby('destination')['aircraft_code'].apply(list).to_dict()
ai_mp['PMI']

['07be2e8d', '280c983e', 'cad7000e', '90455e0e']

In [ ]:
cursor = conn.cursor()

for i in range(len(df_fl)):
    destination = df_fl.at[i, 'destination']
    
    if destination in list(ai_mp.keys()):
        # Get random aircraft for this destination
        aircraft_code = random.choice(ai_mp[destination])
        
        # Assuming you have a flight_id column to update specific flight
        if 'flight_id' in df_fl.columns:
            flight_id = df_fl.at[i, 'flight_id']
            cursor.execute(
                """UPDATE travel.flights SET aircraft_code = %s WHERE flight_id = %s""",
                (aircraft_code, flight_id)
            )
        
# Commit once after all updates
conn.commit()
cursor.close()

In [87]:
conn.commit()
cursor.close()

In [92]:
cursor = conn.cursor()

# for i in range(len(df_c)):
    # if 'customer_id' in df_c.columns:
        # customer_id = df_c.at[i, 'customer_id']
cursor.execute("""UPDATE travel.customers SET name = %s WHERE name = %s""",
('Gills Stone', 'Nacho Garrón'))

conn.commit()
cursor.close()

In [8]:
cursor = conn.cursor()

for i in range(len(df_b)):
    destination = df_b.at[i, 'length_of_stay']
    
    los = random.choice(range(3,8))

    # Assuming you have a booking_id column to update specific booking
    if 'booking_id' in df_b.columns:
        booking_id = df_b.at[i, 'booking_id']
        cursor.execute(
        """UPDATE travel.bookings SET length_of_stay = %s WHERE booking_id = %s""",
        (los, booking_id)
        )
        
conn.commit()
cursor.close()

In [8]:
cursor = conn.cursor()

for i in range(len(df_hp)):
    days = ['2026-01-20', '2026-01-21', '2026-01-22', '2026-01-23',
           '2026-01-24', '2026-01-25', '2026-01-26', '2026-01-27']    
    price = random.uniform(80, 300)

    # Assuming you have a rate_plan_id column to update specific hotel price
    if 'rate_plan_id' in df_hp.columns:
        rate_plan_id = df_hp.at[i, 'rate_plan_id']
        cursor.execute(
        """UPDATE travel.hotel_prices SET date = %s, price_per_night = %s WHERE rate_plan_id = %s""",
        (random.choice(days), price, rate_plan_id)
        )
        
conn.commit()
cursor.close()

In [ ]:
conn.commit()
cursor.close()



In [5]:
random.choice(range(3, 7))

4

In [38]:
HOTELS

[('3aaf0cc6',
  'Hotel Centro',
  'UrbanStay',
  'Madrid',
  'Spain',
  40.4168,
  -3.7038,
  4,
  [120, 250]),
 ('4c9692b3',
  'Hotel Mar',
  'CoastInn',
  'Barcelona',
  'Spain',
  41.3851,
  2.1734,
  4,
  [140, 280]),
 ('1ad4f8c1',
  'Le Meurice',
  'Hotels',
  'Paris',
  'France',
  48.8658,
  2.3275,
  5,
  [600, 1500]),
 ('86a8b017',
  'Hotel Cap Rocat',
  'Hotel',
  'Palma',
  'Spain',
  39.5331,
  2.5819,
  5,
  [400, 900]),
 ('a6065b00',
  'The Plaza',
  'FamousHotels',
  'New York',
  'USA',
  40.7644,
  -73.9742,
  5,
  [800, 2000]),
 ('fc55b3c4',
  'Burj Al Arab Jumeirah',
  'LuxuryStays',
  'Dubai',
  'UAE',
  25.1412,
  55.1853,
  5,
  [1200, 3000]),
 ('8d1f74d7',
  'Los Tajibos Hotel',
  'Hotel',
  'Santa Cruz',
  'Bolivia',
  -17.7756,
  -63.1974,
  3,
  [70, 150]),
 ('e5d181db',
  'The Savoy',
  'Hotel',
  'London',
  'United Kingdom',
  51.5105,
  -0.1207,
  5,
  [450, 1200])]

In [74]:
random.choice(range(HOTELS[1][-1][0],HOTELS[1][-1][1]))

209

In [59]:
HOTELS[1][-1]

[140, 280]

In [56]:
random.choice(range(3, 7))

3

In [36]:
df_b.head()

,booking_id,customer_id,hotel_id,booking_type,status,confirmation_code,currency,total_amount,created_at,length_of_stay
0,5704c879,eaa0defc,3aaf0cc6,HOTEL,TICKETED,PNR0E3046,EUR,250.00,2026-01-22 20:00:17.096942+01:00,3
1,8d0f4cef,cae6a133,3aaf0cc6,AIR,CANCELLED,PNR43862F,EUR,250.00,2026-01-22 20:00:17.097043+01:00,7
2,cf601636,d40b3a36,3aaf0cc6,HOTEL,CANCELLED,PNRE7E9D5,EUR,250.00,2026-01-22 20:00:17.097451+01:00,3
3,f7d9385f,e1118880,3aaf0cc6,HOTEL,PENDING_PAYMENT,PNRB69637,EUR,250.00,2026-01-22 20:00:17.097496+01:00,4
4,4d28e7cb,6b67a717,3aaf0cc6,HOTEL,PENDING_PAYMENT,PNR024B06,EUR,250.00,2026-01-22 20:00:17.097617+01:00,4


In [84]:
# random.choice(range(HOTELS[j][-1][0],HOTELS[j][-1][1]))
random.choice(range(HOTELS[j][-1][0],HOTELS[j][-1][1]))

203

In [89]:
cursor = conn.cursor()

for i in range(len(df_b)):
    book_id = df_b.at[i, 'booking_id']
    for j in range(len(HOTELS)):
        
        cursor.execute(
        """UPDATE travel.bookings SET total_amount = %s WHERE hotel_id = %s 
        and booking_id = %s""", 
        (random.choice(range(HOTELS[j][-1][0],HOTELS[j][-1][1])), 
        HOTELS[j][0], book_id))
        
conn.commit()
cursor.close()

In [90]:
cursor = conn.cursor()

for i in range(len(df_hp)):
    # Assuming you have a rate_plan_id column to update specific hotel price
    rate_plan_id = df_hp.at[i, 'rate_plan_id']

    for j in range(len(HOTELS)):

        cursor.execute(
        """UPDATE travel.hotel_prices SET price_per_night = %s 
        WHERE rate_plan_id = %s and hotel_id = %s""",
        (random.choice(range(HOTELS[j][-1][0],HOTELS[j][-1][1])), 
         rate_plan_id, HOTELS[j][0]))
        
conn.commit()
cursor.close()

In [9]:
travels_from_mad

[('MAD',
  'Adolfo Suárez Madrid–Barajas',
  'BCN',
  'Barcelona–El Prat',
  504,
  '1h 10m',
  [60, 180]),
 ('MAD',
  'Adolfo Suárez Madrid–Barajas',
  'CDG',
  'Charles de Gaulle',
  1053,
  '2h 5m',
  [90, 250]),
 ('MAD',
  'Adolfo Suárez Madrid–Barajas',
  'LHR',
  'Heathrow',
  1265,
  '2h 20m',
  [100, 280]),
 ('MAD',
  'Adolfo Suárez Madrid–Barajas',
  'PMI',
  'Palma de Mallorca Airport',
  553,
  '1h 15m',
  [70, 200]),
 ('MAD',
  'Adolfo Suárez Madrid–Barajas',
  'JFK',
  'John F. Kennedy International',
  5770,
  '8h 30m',
  [450, 1200]),
 ('MAD',
  'Adolfo Suárez Madrid–Barajas',
  'DXB',
  'Dubai International Airport',
  5610,
  '7h 10m',
  [400, 1100]),
 ('MAD',
  'Adolfo Suárez Madrid–Barajas',
  'SCZ',
  'Santa Cruz de la Sierra Airport',
  8850,
  '10 30m',
  [700, 1800])]

In [10]:
df_p.head(2)

,price_id,inventory_id,currency,base_fare,taxes,total_fare,valid_until_ts
0,c9e3484d,7f603ddd,EUR,141.89,29.80,171.69,2026-01-22 22:30:01.254317+01:00
1,bb324741,1a756767,EUR,208.03,43.69,251.72,2026-01-22 22:30:01.254355+01:00


In [30]:
171.69-171.69*.15 

145.9365

In [28]:
for j in range(len(travels_from_mad)):
    print(random.choice(range(travels_from_mad[j][-1][0],travels_from_mad[j][-1][1])),
          travels_from_mad[j][2])

117 BCN
233 CDG
219 LHR
185 PMI
970 JFK
1001 DXB
997 SCZ


In [ ]:
("""
UPDATE travel.prices p
    SET base_fare = %s, taxes = %s, total_fare = %s
    WHERE p.price_id = %s 
    AND p.price_id IN (
        SELECT bi.price_id
        FROM travel.booking_items bi
        JOIN travel.flights f ON bi.flight_id = f.flight_id
        WHERE f.destination = %s
    )"""),
    (base_price, taxes, total_price, price_id, )

In [38]:
cursor = conn.cursor()

for i in range(len(df_p)):
    
    # Assuming you have a rate_plan_id column to update specific hotel price
    price_id = df_p.at[i, 'price_id']

    for j in range(len(travels_from_mad)):
        total_fare = random.choice(range(travels_from_mad[j][-1][0],travels_from_mad[j][-1][1]))
        taxes = round(total_fare * 0.15, 2)
        base_fare = total_fare - taxes

        cursor.execute( """
        UPDATE travel.prices p
        SET base_fare = %s, taxes = %s, total_fare = %s
        WHERE p.price_id = %s 
        AND p.price_id IN (
        SELECT bi.price_id
        FROM travel.booking_items bi
        JOIN travel.flights f ON bi.flight_id = f.flight_id
        WHERE f.destination = %s
        )""",
        (base_fare, taxes, total_fare, price_id, travels_from_mad[j][2]))
        
conn.commit()
cursor.close()

In [37]:
for i in range(len(df_p)):
    
    # Assuming you have a rate_plan_id column to update specific hotel price
    price_id = df_p.at[i, 'price_id']

    for j in range(len(travels_from_mad)):
        total_fare = random.choice(range(travels_from_mad[j][-1][0],travels_from_mad[j][-1][1]))
        taxes = round(total_fare * 0.15, 2)
        base_fare = total_fare - taxes

        print("""
        UPDATE travel.prices p
        SET base_fare = %s, taxes = %s, total_fare = %s
        WHERE p.price_id = %s 
        AND p.price_id IN (
        SELECT bi.price_id
        FROM travel.booking_items bi
        JOIN travel.flights f ON bi.flight_id = f.flight_id
        WHERE f.destination = %s
        )""",
        (base_fare, taxes, total_fare, price_id, travels_from_mad[j][2]))


        UPDATE travel.prices p
        SET base_fare = %s, taxes = %s, total_fare = %s
        WHERE p.price_id = %s 
        AND p.price_id IN (
        SELECT bi.price_id
        FROM travel.booking_items bi
        JOIN travel.flights f ON bi.flight_id = f.flight_id
        WHERE f.destination = %s
        ) (77.35, 13.65, 91, 'c9e3484d', 'BCN')

        UPDATE travel.prices p
        SET base_fare = %s, taxes = %s, total_fare = %s
        WHERE p.price_id = %s 
        AND p.price_id IN (
        SELECT bi.price_id
        FROM travel.booking_items bi
        JOIN travel.flights f ON bi.flight_id = f.flight_id
        WHERE f.destination = %s
        ) (204.85, 36.15, 241, 'c9e3484d', 'CDG')

        UPDATE travel.prices p
        SET base_fare = %s, taxes = %s, total_fare = %s
        WHERE p.price_id = %s 
        AND p.price_id IN (
        SELECT bi.price_id
        FROM travel.booking_items bi
        JOIN travel.flights f ON bi.flight_id = f.flight_id
        WHERE f.destina